In [2]:
#Spark Streaming integeration with kafka 0.10.0.0 and above, is still in experimental status, Hence using Kafka 0.9
#(http://spark.apache.org/docs/latest/streaming-kafka-integration.html)
#Run Zookeeper to maintain Kafka, command to be run from Kafka root dir
#change directory to kafka directory

#cd /usr/hdp/2.4.0.0-169/kafka
#bin/zookeeper-server-start.sh config/zookeeper.properties

SyntaxError: invalid syntax (<ipython-input-2-5c019b67df78>, line 5)

In [ ]:
#Start Kafka server, aditional servers can be added as per requirement.
#bin/kafka-server-start.sh config/server.properties

In [6]:
#Create Producer.py

from kafka import KafkaClient, SimpleProducer
import json,requests

kafka = KafkaClient('localhost:9092')

producer = SimpleProducer(kafka)

r = requests.get("https://stream.meetup.com/2/rsvps",stream=True)

for line in r.iter_lines():
    producer.send_messages('meetup',line)
    print type(line)

kafka.close()

KafkaUnavailableError: KafkaUnavailableError: All servers failed to process request: [('127.0.0.1', 9092, 2)]

In [ ]:
#Create Consumer.py file
from kafka import KafkaConsumer

consumer = KafkaConsumer('meetup', group_id = '1', bootstrap_servers = ['localhost:9092'])

for message in consumer:
    print message.value

In [ ]:
#Create spark_meetup.py file
from __future__ import print_function
import json
import sys
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

f_count = 0;

def country_count_update(newValue, existingCount):
    if existingCount is None:
        existingCount = 0

    return sum(newValue)+existingCount

if __name__ == "__main__":

    if len(sys.argv) != 3:
        print("Usage: spark_meetup.py <zk> <topic>", file=sys.stderr)
        exit(-1)

    sc = SparkContext("local[2]",appName="PythonStreamingMeetup")
    sc.setLogLevel("WARN")
    ssc = StreamingContext(sc, 1)
    ssc.checkpoint("checkpoint")

    zkQuorum, topic = sys.argv[1:]
    kvs = KafkaUtils.createStream(ssc, zkQuorum, "1", {topic: 1})
    rsvps = kvs.map(lambda x: x[1])
    rsvps_json = rsvps.map(lambda x: json.loads(x.encode('ascii','ignore')))
    rsvps_us = rsvps_json.filter(lambda x: x['group']['group_country']=='us')
    rsvps_city_pair = rsvps_us.map(lambda x: (x['group']['group_city'],1))
    rsvps_city_statefulCount = rsvps_city_pair.updateStateByKey(country_count_update)
    
    rsvps_city_statefulCount.pprint()

    ssc.start()
ssc.awaitTermination()

In [ ]:
#Start Producer.py to start reading data from the meetup stream and store it in '''meetup''' kafka topic.
#./Producer.py

In [ ]:
#Start Consumer.py to consume the stream from the '''meetup''' topic
#./Consumer.py

In [ ]:
#Submit the spark job spark_meetup.py, to read the data into Spark Streaming from Kafka.
#bin/spark-submit --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.1 spark_meetup.py localhost:2181 meetup